In [31]:
import pickle
import os
import glob
import collections
import json
from tqdm import tqdm as tqdm

meta_cache_dir='/opt/gitserial/tianchi/cache'
train_meta_cache_file='train_image_annotation.pkl'
val_meta_cache_file='val_image_annotation.pkl'
train_video_meta_cache_file='video_image_annotation.pkl'
trainval_pos_pair_dict=collections.OrderedDict()
for k in train_ann.keys():
    trainval_pos_pair_dict[k]={'file_pair':[]}
    
with open(os.path.join(meta_cache_dir,train_meta_cache_file),'rb') as fid:
    train_ann = pickle.load(fid)
with open(os.path.join(meta_cache_dir,val_meta_cache_file),'rb') as fid:
    val_ann = pickle.load(fid)
# with open(os.path.join(meta_cache_dir,train_video_meta_cache_file),'rb') as fid:
#     train_video_ann = pickle.load(fid)
'''save file_name' and 'file_name's pair
    {
        'i_123_1321.jpg':[i_124_131.jpg, i_213_213.jpg, v_213_1231,jpg ...],
        '1243412':[],
        ...
    }
'''

"save file_name' and 'file_name's pair\n    {\n        'i_123_1321.jpg':[i_124_131.jpg, i_213_213.jpg, v_213_1231,jpg ...],\n        '1243412':[],\n        ...\n    }\n"

In [344]:
video_annotation_files[-1]

'/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/029835.json'

In [43]:
root_dir='/opt/gitserial/tianchi'
cache_file='video_instance_dict.pkl'
data_paths=sorted(glob.glob(os.path.join(root_dir,'data/')+'train_dataset_part?'))
video_instance_dict=collections.OrderedDict()

video_annotation_files=[]
for x in data_paths:
    video_annotation_files.extend(glob.glob(x+'/video_annotation/*.json'))
print(video_annotation_files[:10])

['/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/104588.json', '/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/026748.json', '/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/093713.json', '/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/025117.json', '/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/030888.json', '/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/117067.json', '/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/001218.json', '/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/004369.json', '/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/006828.json', '/opt/gitserial/tianchi/data/train_dataset_part1/video_annotation/025937.json']


In [64]:
video_instance_dict=collections.OrderedDict()
instances=set()
for i in instances:
    video_instance_dict[i]=[]
for file in tqdm(video_annotation_files):
    with open(file,'r') as fid:
        obj = json.load(fid)
        for v in obj['frames'] :
            annotations=v['annotations']
            for annotation in annotations:
                instance_id = annotation['instance_id']
                instances.add(f'{instance_id}')


100%|██████████| 30000/30000 [00:01<00:00, 16187.34it/s]


In [348]:
video_annotation_files[-20:-1]


['/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/043089.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/085043.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/055298.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/119996.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/029935.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/119502.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/047795.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/028424.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/005397.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/053475.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/072273.json',
 '/opt/gitserial/tianchi/data/train_dataset_part6/video_annotation/066891.json',
 '/opt/gitserial/tianchi/dat

In [351]:
video_instance_dict['24308901']

['v_043089_0',
 'v_043089_40',
 'v_043089_80',
 'v_043089_120',
 'v_043089_160',
 'v_043089_200',
 'v_043089_320',
 'v_043089_360']

In [322]:
video_instance_dict={k:[] for k in instances}

In [352]:
len(video_instance_dict.keys())

64508

In [326]:
for file in tqdm(video_annotation_files):
    with open(file,'r') as fid:
        obj = json.load(fid)
        video_id = obj['video_id']
        for v in obj['frames'] :
            annotations=v['annotations']
            frame_index=v['frame_index']
            for annotation in annotations:
                instance_id = annotation['instance_id']
                xx=video_instance_dict[f'{instance_id}']
                xx.append(f'v_{video_id}_{frame_index}')
#                 print(xx)
                video_instance_dict[instance_id]=xx
#     break

100%|██████████| 30000/30000 [00:02<00:00, 14371.62it/s]


In [331]:
for v in train_ann.values():
    count=0
    for i in v:
        file_name= i['file_name']
        train_dir_num=os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(file_name))))
        trainval_pos_pair_dict[item_id]['train_dir_num']=f'{train_dir_num}'
        file_pair=set()
        item_id=i['annotation']['item_id']
        for ann in i['annotation']['annotations']:
            viewpoint=ann['viewpoint']
            if count == 0 and viewpoint > 0:
                count+=1
                continue
            instance_id=ann['instance_id']
            if instance_id > 0:
                v_file_list=video_instance_dict.get(f'{instance_id}')
                if not v_file_list:
                    continue
                file_pair.update(v_file_list)
                file_pair.add(f'i_{item_id}_{os.path.basename(file_name)}')
            trainval_pos_pair_dict[item_id]['file_pair']=sorted(file_pair)
#                 print(item_id,train_dir_num,file_pair)
        
#         break
#     break

In [315]:
trainval_pos_pair_dict=collections.OrderedDict()
for k in train_ann.keys():
    trainval_pos_pair_dict[k]={'file_pair':[]}

052206


In [368]:
# trainval_pos_pair_dict

In [366]:
# trainval_pos_pair_dict
with open(os.path.join(meta_cache_dir,'trainval_pos_pair_dict.pkl'),'wb') as fid:
    pickle.dump(trainval_pos_pair_dict,fid,pickle.HIGHEST_PROTOCOL)